In [1]:
from youtube_transcript_api import YouTubeTranscriptApi


In [2]:
def get_youtube_transcript(video_url: str) -> list:
    video_id = video_url.split("v=")[1]
    transcript = YouTubeTranscriptApi.get_transcript(video_id)
    return transcript
video_url = "https://www.youtube.com/watch?v=8QfI5a7lTKU"
video_id = video_url.split("v=")[1]

transcript = YouTubeTranscriptApi.get_transcript(video_id)

for segment in transcript:
    print(segment['text'])

what is the common point between all
those robots
well all the robots you see here are
or can be powered by rouse
roth means robot operating system
it is something between a middleware
and a framework built especially for
robotics applications rao's goal
is to provide a standard for robotic
software
that developers can use and reuse in
any robot once you know how to use ros
efficiently you will be able to set up
a new software for a new robot in no
time
you first need to master the basics and
core functionalities
which you will see in this course and
then
for every new robot you program you will
gain more skills
that can apply to other robots
the more you know about ross the easiest
it will be for you to program and
understand
the code of any robot
but is that it no another
philosophy behind ross is don't reinvent
the wheel reinventing the wheel is a
very common problem
in robotics people always start building
new software
that is already existing and thus they
don't have the time
to w

In [3]:
## MongoDB
from pymongo import MongoClient

# MongoDB connection
client = MongoClient("mongodb://localhost:27017/")  # Replace with your MongoDB connection string
db = client["youtube_database"]  # Database name
collection = db["transcripts"]   # Collection name

# Insert into MongoDB
document = {
    "source": "youtube",
    "video_id": video_id,
    "transcript": transcript
}

collection.insert_one(document)
print("Transcript stored into MongoDB")

Transcript stored into MongoDB


In [4]:
result = collection.find_one({"video_id": video_id})
print(result.keys())

dict_keys(['_id', 'video_id', 'transcript'])


In [5]:
# Extract text from transcripts
sentences = [i['text'] for i in result['transcript']]
print(sentences)







['what is the common point between all', 'those robots', 'well all the robots you see here are', 'or can be powered by rouse', 'roth means robot operating system', 'it is something between a middleware', 'and a framework built especially for', "robotics applications rao's goal", 'is to provide a standard for robotic', 'software', 'that developers can use and reuse in', 'any robot once you know how to use ros', 'efficiently you will be able to set up', 'a new software for a new robot in no', 'time', 'you first need to master the basics and', 'core functionalities', 'which you will see in this course and', 'then', 'for every new robot you program you will', 'gain more skills', 'that can apply to other robots', 'the more you know about ross the easiest', 'it will be for you to program and', 'understand', 'the code of any robot', 'but is that it no another', "philosophy behind ross is don't reinvent", 'the wheel reinventing the wheel is a', 'very common problem', 'in robotics people always

In [6]:
# Chunk sentences together for embedding
#TODO: Make this chunking better:Maybe capitalize and fins end of sentences in a better way
sentences_in_chunk = 6
chunks = []
for i in range(0, len(sentences), sentences_in_chunk):
    chunks.append(". ".join(sentences[i:i + sentences_in_chunk]))

print(len(chunks))

28


In [7]:
# Embed these chunks
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('paraphrase-MiniLM-L6-v2') # TODO: Pick better models

embeddings = model.encode(chunks)
embeddings_dict = dict(zip(chunks, embeddings))
print(embeddings.shape)


(28, 384)


In [8]:
# Store embeddings in MongoDB
# Insert into MongoDB
collection = db["embeddings"]   # Collection name
document = {
    "source": "youtube",
    "video_id": video_id,
    "embeddings": embeddings.tolist()
}

collection.insert_one(document)
print("Embeddings stored into MongoDB")


Embeddings stored into MongoDB


In [9]:
# TODO: Do further preprocessing to tokens

In [10]:
## Similarity search using qdrant
from qdrant_client import QdrantClient
from qdrant_client.http import models
from torchvision.transforms import Resize
from tqdm import tqdm
import torch

qdrant = QdrantClient(":memory:") # Create in-memory Qdrant instance, for testing, CI/CD

#Create a collection or database of texts where you store the embeddings
my_collection = "text_collection"

qdrant.create_collection(
    collection_name=my_collection,
    vectors_config=models.VectorParams(size=384, distance=models.Distance.COSINE)
)

# Insert embeddings into Qdrant
for i, (chunk, embedding) in enumerate(zip(chunks, embeddings)):
    qdrant.upsert(
        collection_name=my_collection,
        points=[models.PointStruct(
            id=i,
            vector=embedding,
            payload={"text": chunk}
        )]
    )






In [11]:
qdrant.count(
    collection_name=my_collection,
    exact=True,
)

CountResult(count=28)

In [12]:
## Sanity test
sample_sentence = "What is the meaning of life?"
sample_embedding = model.encode(sample_sentence)
search_result = qdrant.search(
    collection_name=my_collection,
    query_vector=sample_embedding,
    limit=5
)

In [13]:
search_result

[ScoredPoint(id=21, version=0, score=0.15166017413139343, payload={'text': "and great libraries that's not all. ros is said to be language agnostic. it means that you can program some parts. of your application. in a programming language and another. part"}, vector=None, shard_key=None, order_value=None),
 ScoredPoint(id=10, version=0, score=0.07326250523328781, payload={'text': 'create a robot software. which needs a lot of communication. between its. sub-programs or has some functionalities. that go beyond a very simple use case. etc so for a robot'}, vector=None, shard_key=None, order_value=None),
 ScoredPoint(id=0, version=0, score=0.061092559248209, payload={'text': 'what is the common point between all. those robots. well all the robots you see here are. or can be powered by rouse. roth means robot operating system. it is something between a middleware'}, vector=None, shard_key=None, order_value=None),
 ScoredPoint(id=22, version=0, score=0.05160965770483017, payload={'text': "in

# LLM

In [14]:
# Get GPU available memory
import torch
gpu_memory_bytes = torch.cuda.get_device_properties(0).total_memory
gpu_memory_gb = round(gpu_memory_bytes / (2**30))
print(f"Available GPU memory: {gpu_memory_gb} GB")

Available GPU memory: 4 GB


In [15]:
from huggingface_hub import login
login()

Token has not been saved to git credential helper.


In [33]:
use_quantization_config = False
model_id = "gpt2"

In [34]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
from transformers.utils import is_flash_attn_2_available 

# 1. Create quantization config for smaller model loading (optional)
# Requires !pip install bitsandbytes accelerate, see: https://github.com/TimDettmers/bitsandbytes, https://huggingface.co/docs/accelerate/
# For models that require 4-bit quantization (use this if you have low GPU memory available)
from transformers import BitsAndBytesConfig
quantization_config = BitsAndBytesConfig(load_in_4bit=True,
                                         bnb_4bit_compute_dtype=torch.float16)

# Bonus: Setup Flash Attention 2 for faster inference, default to "sdpa" or "scaled dot product attention" if it's not available
# Flash Attention 2 requires NVIDIA GPU compute capability of 8.0 or above, see: https://developer.nvidia.com/cuda-gpus
# Requires !pip install flash-attn, see: https://github.com/Dao-AILab/flash-attention 
if (is_flash_attn_2_available()) and (torch.cuda.get_device_capability(0)[0] >= 8):
  attn_implementation = "flash_attention_2"
else:
  attn_implementation = "sdpa"
print(f"[INFO] Using attention implementation: {attn_implementation}")

# 2. Pick a model we'd like to use (this will depend on how much GPU memory you have available)
# model_id = "google/gemma-7b-it"
model_id = model_id # (we already set this above)
print(f"[INFO] Using model_id: {model_id}")

# 3. Instantiate tokenizer (tokenizer turns text into numbers ready for the model) 
tokenizer = AutoTokenizer.from_pretrained(pretrained_model_name_or_path=model_id)

# 4. Instantiate the model
# llm_model = AutoModelForCausalLM.from_pretrained(pretrained_model_name_or_path=model_id, 
#                                                  torch_dtype=torch.float16, # datatype to use, we want float16
#                                                  quantization_config=quantization_config if use_quantization_config else None,
#                                                  low_cpu_mem_usage=False, # use full memory 
#                                                  attn_implementation=attn_implementation) # which attention version to use
llm_model = AutoModelForCausalLM.from_pretrained(pretrained_model_name_or_path=model_id)

if not use_quantization_config: # quantization takes care of device setting automatically, so if it's not used, send model to GPU 
    llm_model.to("cuda")

[INFO] Using attention implementation: sdpa
[INFO] Using model_id: gpt2


In [35]:
llm_model.eval() # put model in evaluation mode
llm_model

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2SdpaAttention(
          (c_attn): Conv1D(nf=2304, nx=768)
          (c_proj): Conv1D(nf=768, nx=768)
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D(nf=3072, nx=768)
          (c_proj): Conv1D(nf=768, nx=3072)
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=50257, bias=False)
)

In [36]:
def get_model_num_params(model: torch.nn.Module):
    return sum([param.numel() for param in model.parameters()])

get_model_num_params(llm_model)

124439808

In [37]:
def get_model_mem_size(model: torch.nn.Module):
    """
    Get how much memory a PyTorch model takes up.

    See: https://discuss.pytorch.org/t/gpu-memory-that-model-uses/56822
    """
    # Get model parameters and buffer sizes
    mem_params = sum([param.nelement() * param.element_size() for param in model.parameters()])
    mem_buffers = sum([buf.nelement() * buf.element_size() for buf in model.buffers()])

    # Calculate various model sizes
    model_mem_bytes = mem_params + mem_buffers # in bytes
    model_mem_mb = model_mem_bytes / (1024**2) # in megabytes
    model_mem_gb = model_mem_bytes / (1024**3) # in gigabytes

    return {"model_mem_bytes": model_mem_bytes,
            "model_mem_mb": round(model_mem_mb, 2),
            "model_mem_gb": round(model_mem_gb, 2)}

get_model_mem_size(llm_model)

{'model_mem_bytes': 510342192, 'model_mem_mb': 486.7, 'model_mem_gb': 0.48}

In [38]:
input_text = "What is ros?"
print(f"Input text:\n{input_text}")

# Tokenize the input text
input_ids = tokenizer(input_text, return_tensors="pt").input_ids
print(f"Input IDs:\n{input_ids}")

prompt = input_text #TODO: Fix a prompt structure?

Input text:
What is ros?
Input IDs:
tensor([[2061,  318,  686,   82,   30]])


In [39]:
# Tokenize the input text (turn it into numbers) and send it to GPU
input_ids = tokenizer(prompt, return_tensors="pt").to("cuda")
print(f"Model input (tokenized):\n{input_ids}\n")

# Generate outputs passed on the tokenized input
# See generate docs: https://huggingface.co/docs/transformers/v4.38.2/en/main_classes/text_generation#transformers.GenerationConfig 
outputs = llm_model.generate(**input_ids,
                             max_new_tokens=256) # define the maximum number of new tokens to create
print(f"Model output (tokens):\n{outputs[0]}\n")

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


Model input (tokenized):
{'input_ids': tensor([[2061,  318,  686,   82,   30]], device='cuda:0'), 'attention_mask': tensor([[1, 1, 1, 1, 1]], device='cuda:0')}

Model output (tokens):
tensor([ 2061,   318,   686,    82,    30,   198,   198,    49,  4629,   389,
          257,  2099,   286, 17182,   973,   284,  1745,   257, 17182,    13,
         1119,   389,   973,   284,  1745,   257, 17182,   287,  1295,    11,
          393,   284,  1745,   257, 17182,   287,  1295,   319,   257,  3704,
          286,  4898,    13,  1119,   389,   635,   973,   284,  1745,   257,
        17182,   287,  1295,   319,   257,  3704,   286,  4898,    13,   198,
          198,    49,  4629,   389,   973,   284,  1745,   257, 17182,   287,
         1295,   319,   257,  3704,   286,  4898,    13,  1119,   389,   635,
          973,   284,  1745,   257, 17182,   287,  1295,   319,   257,  3704,
          286,  4898,    13,   198,   198,    49,  4629,   389,   973,   284,
         1745,   257, 17182,   287, 

In [40]:
# Decode the output tokens to text
outputs_decoded = tokenizer.decode(outputs[0])
print(f"Model output (decoded):\n{outputs_decoded}\n")

Model output (decoded):
What is ros?

Roses are a type of rope used to hold a rope. They are used to hold a rope in place, or to hold a rope in place on a piece of wood. They are also used to hold a rope in place on a piece of wood.

Roses are used to hold a rope in place on a piece of wood. They are also used to hold a rope in place on a piece of wood.

Roses are used to hold a rope in place on a piece of wood. They are also used to hold a rope in place on a piece of wood.

Roses are used to hold a rope in place on a piece of wood. They are also used to hold a rope in place on a piece of wood.

Roses are used to hold a rope in place on a piece of wood. They are also used to hold a rope in place on a piece of wood.

Roses are used to hold a rope in place on a piece of wood. They are also used to hold a rope in place on a piece of wood.

Roses are used to hold a rope in place on a piece of wood. They are also used to hold a rope in place on a piece of



In [41]:
print(f"Input text: {input_text}\n")
print(f"Output text:\n{outputs_decoded.replace(prompt, '').replace('<bos>', '').replace('<eos>', '')}")

Input text: What is ros?

Output text:


Roses are a type of rope used to hold a rope. They are used to hold a rope in place, or to hold a rope in place on a piece of wood. They are also used to hold a rope in place on a piece of wood.

Roses are used to hold a rope in place on a piece of wood. They are also used to hold a rope in place on a piece of wood.

Roses are used to hold a rope in place on a piece of wood. They are also used to hold a rope in place on a piece of wood.

Roses are used to hold a rope in place on a piece of wood. They are also used to hold a rope in place on a piece of wood.

Roses are used to hold a rope in place on a piece of wood. They are also used to hold a rope in place on a piece of wood.

Roses are used to hold a rope in place on a piece of wood. They are also used to hold a rope in place on a piece of wood.

Roses are used to hold a rope in place on a piece of wood. They are also used to hold a rope in place on a piece of


In [43]:
# Nutrition-style questions generated with GPT4
gpt4_questions = [
    "WWhat are the main differences between ROS 1 and ROS 2?",
    "Explain the concept of topics, services, and actions in ROS.",
    "What is the purpose of the catkin_make command in ROS 1?",
    "What is the function of rviz in ROS?",
    "What is the purpose of rqt_graph, and how can it help in debugging a ROS system?"
] 

# Manually created question list
manual_questions = [
    "What is ros?",
]

query_list = gpt4_questions + manual_questions

In [44]:
import random
query = random.choice(query_list)

print(f"Query: {query}")



Query: WWhat are the main differences between ROS 1 and ROS 2?


In [45]:
search_result = qdrant.search(
    collection_name=my_collection,
    query_vector=model.encode(query),
    limit=5
)

In [47]:
search_result

[ScoredPoint(id=21, version=0, score=0.5012298822402954, payload={'text': "and great libraries that's not all. ros is said to be language agnostic. it means that you can program some parts. of your application. in a programming language and another. part"}, vector=None, shard_key=None, order_value=None),
 ScoredPoint(id=24, version=0, score=0.4134202003479004, payload={'text': "powered by ros. using python and c plus plus. ros is also open source with an. active and growing community so. you can directly see what's going on you. can get help"}, vector=None, shard_key=None, order_value=None),
 ScoredPoint(id=7, version=0, score=0.4084433913230896, payload={'text': "so this is the why behind ross. now when to use ros maybe. you've already started to program robots. with. an arduino board with a custom code. on a computer and what you will quickly"}, vector=None, shard_key=None, order_value=None),
 ScoredPoint(id=1, version=0, score=0.2859797477722168, payload={'text': "and a framework bu

## Augment Similarity Search and LLM output 